## **RADI608: Data Mining and Machine Learning**

### Assignment: APriori Algorithm
**Romen Samuel Rodis Wabina** <br>
Student, PhD Data Science in Healthcare and Clinical Informatics <br>
Clinical Epidemiology and Biostatistics, Faculty of Medicine (Ramathibodi Hospital) <br>
Mahidol University

Note: In case of Python Markdown errors, you may access the assignment through this GitHub [Link](https://github.com/rrwabina/RADI608/tree/main/Submitted)

### Question 1: Generate the frequent itemset and association rules using the given transaction database below. Use a <code>minimum support threshold = 50%</code>

<center>
<img src = "figures/question1.jpg" width = '650'> <br>
</center>

##### We find the support of each item to determine which items are frequent. This is shown in the figure below.

<center>
<img src = "figures/02.jpg" width = '650'> <br>
</center>

##### We then used an initial minimum support threshold (i.e., pruning) and applied to find all frequent itemsets in the database.

<center>
<img src = "figures/03.jpg" width = '650'> <br>
</center>

<center>
<img src = "figures/04.jpg" width = '650'> <br>
</center>

<center>
<img src = "figures/05.jpg" width = '650'> <br>
</center>

### Question 2: Using the <code>prescriptionDB.csv</code>, perform an Apriori Algorithm to generate the association rules by using <code>support = 0.001, confidence = 0.5,</code> and find the top 10 of the <code>RHS = 'OMPZ'</code>.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

import mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

from sklearn.preprocessing import LabelEncoder
import warnings 
warnings.filterwarnings('ignore')

In [72]:
df = pd.read_csv('../data/prescriptionDB.csv')
df.set_index('Item ID')
df.fillna('', inplace = True)
df = df[['Code.1', 'Code.2', 'Code.3', 'Code.4']]
dataset = df.to_numpy()

encoder = TransactionEncoder()
encoder_array = encoder.fit(dataset).transform(dataset)
df_encode = pd.DataFrame(encoder_array, columns = encoder.columns_)

def encode_units(x):
    if x is False:
        return 0
    if x is True:
        return 1

frequent_itemsets = apriori(df_encode, min_support = 0.001, use_colnames = True)
rules = association_rules(frequent_itemsets, metric = 'lift', min_threshold = 1)

def cariIndex(setList, data2):
   i=-1
   for data in setList:
       i=i+1
       if data == data2:
           return i
   return -1 

rule_ompz = rules[rules['consequents'] == frozenset({'OMPZ'})]
rule_ompz = rule_ompz[rule_ompz['confidence'] >= 0.50]
rule_ompz = rule_ompz.sort_values(by=['lift'], ascending = False).head(10)
rule_ompz

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
115,(NAPX),(OMPZ),0.113511,0.746678,0.103821,0.914634,1.224938,0.019065,2.967489
389,"(, NAPX)",(OMPZ),0.113511,0.746678,0.103821,0.914634,1.224938,0.019065,2.967489
77,(ASPT),(OMPZ),0.047619,0.746678,0.042082,0.883721,1.183537,0.006526,2.178571
247,"(, ASPT)",(OMPZ),0.047619,0.746678,0.042082,0.883721,1.183537,0.006526,2.178571
103,(INDM),(OMPZ),0.008306,0.746678,0.007198,0.866667,1.160697,0.000997,1.899917
351,"(, INDM)",(OMPZ),0.008306,0.746678,0.007198,0.866667,1.160697,0.000997,1.899917
105,(MELO),(OMPZ),0.034330,0.746678,0.029347,0.854839,1.144856,0.003713,1.745109
359,"(, MELO)",(OMPZ),0.034053,0.746678,0.029070,0.853659,1.143276,0.003643,1.731035
100,(IBUP),(OMPZ),0.039037,0.746678,0.031008,0.794326,1.063814,0.001860,1.231670
344,"(, IBUP)",(OMPZ),0.039037,0.746678,0.031008,0.794326,1.063814,0.001860,1.231670


### Question 3: Perform an Apriori Algorithm to generate the association rules by selecting the top 20 rules at <code>support = 0.0001</code> and the LHS has at least two drugs in the basket.

In [71]:
df = pd.read_csv('../data/prescriptionDB.csv')
df.set_index('Item ID')
df.fillna('', inplace = True)
df = df[['Code.1', 'Code.2', 'Code.3', 'Code.4']]
dataset = df.to_numpy()

encoder = TransactionEncoder()
encoder_array = encoder.fit(dataset).transform(dataset)
df_encode = pd.DataFrame(encoder_array, columns = encoder.columns_)

def get_index(setList, data2):
   i = -1
   for data in setList:
       i = i + 1
       if data == data2:
           return i
   return -1 

frequent_itemsets = apriori(df_encode, min_support = 0.0001, use_colnames = True)
rules = association_rules(frequent_itemsets, metric = 'lift', min_threshold = 5)

index1 = []
for idx, value in enumerate(rules['antecedents']):
    if len(value) >= 2:
        if get_index(value, '') != 0:
            index1.append(idx)
index2 = []
for idx, value in enumerate(rules['consequents']):
    if get_index(value, '') != 0:
        index2.append(idx)

index = [value for value in index1 if value in index2]
rules = rules.loc[index].sort_values(by=['lift'], ascending = False).head(20)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
186,"(MELO, XAND)","(OMPZ, PARI)",0.001107,0.001107,0.000277,0.250000,225.750000,0.000276,1.331857
183,"(OMPZ, PARI)","(MELO, XAND)",0.001107,0.001107,0.000277,0.250000,225.750000,0.000276,1.331857
180,"(XAND, OMPZ, PARI)",(MELO),0.000277,0.034330,0.000277,1.000000,29.129032,0.000267,inf
182,"(MELO, OMPZ, XAND)",(PARI),0.000277,0.048173,0.000277,1.000000,20.758621,0.000264,inf
122,"(ULSN, MUCT)",(PARI),0.000277,0.048173,0.000277,1.000000,20.758621,0.000264,inf
107,"(ULSN, GAVI)",(PRVF),0.000277,0.052326,0.000277,1.000000,19.111111,0.000262,inf
112,"(IBUP, XAND)",(PRVF),0.000277,0.052326,0.000277,1.000000,19.111111,0.000262,inf
106,"(ULSN, PRVF)",(GAVI),0.000554,0.030177,0.000277,0.500000,16.568807,0.000260,1.939646
185,"(OMPZ, XAND)","(MELO, PARI)",0.063953,0.000277,0.000277,0.004329,15.636364,0.000259,1.004070
184,"(MELO, PARI)","(OMPZ, XAND)",0.000277,0.063953,0.000277,1.000000,15.636364,0.000259,inf
